In [1]:
# import necessary packages
import os, glob, sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import cv2
import re

from tqdm.auto import tqdm
from ipywidgets import FloatProgress
from scipy import ndimage
from PIL import Image
from importlib import reload

# append path
sys.path.append('../')

# custom
from utils.BuildUNet import UNet
from utils.GetLowestGPU import GetLowestGPU
from utils.TileGenerator import TileGenerator
from utils.Metrics import iou
from utils.GetFileNames import get_file_names
import utils.Traits as Traits
import utils.SegmentImage as SegmentImage
import utils.Measure as Measure

/mnt/DGX01/Personal/krusepi/.venv/lib/python3.9/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.15 (you have 1.4.14). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
# get one example and extract all phenotypes

# paths
image_path = "../data/test/test_images_by_pod/"
mask_path = "../data/test/test_masks_by_pod/"

image_names = get_file_names(image_path, extension=".png")
mask_names = get_file_names(mask_path, extension=".png")


In [3]:
print(image_names[:10])

['1.tt8t x fc6-1.B.WBP-2022.W_0.png', '1.tt8t x fc6-1.B.WBP-2022.W_1.png', '1.tt8t x fc6-1.B.WBP-2022.W_2.png', '1.tt8t x fc6-1.B.WBP-2022.W_3.png', '1.tt8t x fc6-1.B.WBP-2022.W_4.png', '1.tt8t x fc6-1.B.WBP-2022.W_5.png', '1.tt8t x fc6-1.B.WBP-2022.W_6.png', '1.tt8t x fc6-1.B.WBP-2022.W_7.png', '1.tt8t x fc6-1.B.WBP-2022.W_8.png', '1.tt8t x fc6-1.B.WBP-2022.W_9.png']


In [4]:
print(mask_names[:10])

['1.tt8t x fc6-1.B.WBP-2022.W_0.png', '1.tt8t x fc6-1.B.WBP-2022.W_1.png', '1.tt8t x fc6-1.B.WBP-2022.W_2.png', '1.tt8t x fc6-1.B.WBP-2022.W_3.png', '1.tt8t x fc6-1.B.WBP-2022.W_4.png', '1.tt8t x fc6-1.B.WBP-2022.W_5.png', '1.tt8t x fc6-1.B.WBP-2022.W_6.png', '1.tt8t x fc6-1.B.WBP-2022.W_7.png', '1.tt8t x fc6-1.B.WBP-2022.W_8.png', '1.tt8t x fc6-1.B.WBP-2022.W_9.png']


In [5]:
# load example image
test_img = np.array(Image.open(image_path + image_names[0]))
test_mask = np.array(Image.open(mask_path + mask_names[0]))

In [6]:
# make list of all phenotypes to collect
phenotypes = {'seed_count':[], 
              
            # areas
            'seed_area':[],
            'env_area':[],
            'wing_area':[],

            # perimeters
            'seed_perimeter':[],
            'env_perimeter':[],
            'wing_perimeter':[],

            # ...-to-total_area ratios
            'seed_to_total_area':[],
            'env_to_total_area':[],
            'wing_to_total_area':[],

            # ...-to-total_perimeter ratios
            'seed_to_total_perimeter':[],
            'env_to_total_perimeter':[],
            'wing_to_total_perimeter':[],

            # ...-to-seed ratios
            'env_to_seed_area':[],
            'wing_to_seed_area':[],
            'env_to_seed_perimeter':[],
            'wing_to_seed_perimeter':[],

            # ...-to-env ratios
            'wing_to_env_area':[],
            'seed_to_env_area':[],
            'wing_to_env_perimeter':[],
            'seed_to_env_perimeter':[],

            # ...-to-wing ratios
            'seed_to_wing_area':[],
            'env_to_wing_area':[],
            'seed_to_wing_perimeter':[],
            'env_to_wing_perimeter':[],
                
            # color
            'wing_r': [],
            'wing_g': [],
            'wing_b': [],
            'wing_h': [],
            'wing_s': [],
            'wing_v': [],
            'wing_l': [],
            'wing_a': [], 
            'wing_B': [],

            'env_r': [],
            'env_g': [],
            'env_b': [],
            'env_h': [],
            'env_s': [],
            'env_v': [],
            'env_l': [],
            'env_a': [],
            'env_B': [],

            'seed_r': [],
            'seed_g': [],
            'seed_b': [],
            'seed_h': [],
            'seed_s': [],
            'seed_v': [],
            'seed_l': [],
            'seed_a': [],
            'seed_B': []
}

In [7]:
# get areas
wing_area = Traits.area_calc(test_mask[:, :, 0])
env_area = Traits.area_calc(test_mask[:, :, 1:2])
seed_area = Traits.area_calc(test_mask[:, :, 2])

# add to dict
phenotypes['wing_area'].append(wing_area)
phenotypes['env_area'].append(env_area)
phenotypes['seed_area'].append(seed_area)

In [9]:
# get perimeters
wing_p = Traits.perimeter(test_mask)
env_p = Traits.perimeter(test_mask)
seed_p = Traits.perimeter(test_mask)

# add to dict
phenotypes['wing_perimeter'].append(wing_p)
phenotypes['env_perimeter'].append(env_p)
phenotypes['seed_perimeter'].append(seed_p)
